# Preprocessing the mp3 files and metadata to prepare for the dataloader for the MagnaTagATune


In [1]:
# Importing necessary libraries
# TODO: get rid of the tensorflow warnings

import pandas as pd
import numpy as np
from zipfile import ZipFile 
import os
import librosa
pd.options.mode.chained_assignment = None  # default='warn'
from matplotlib import pyplot as plt
from ast import literal_eval
import cv2
import keras
from tensorflow import keras
import tensorflow as tf
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Layer, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
import json
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from collections import OrderedDict
import pyarrow.parquet as pa


2024-08-10 20:16:05.836431: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-10 20:16:05.877071: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 20:16:06.505909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Loading the zipped files and setting up an audio folder in data/magnatagatune

Downloaded the audio data files from: https://mirg.city.ac.uk/codeapps/the-magnatagatune-dataset
Concatenate the 3 files with `cat mp3.zip.* > mp3_all.zip` in the terminal and unzip them with the code below

In [2]:
# '''To unzip the zipped file of mp3 files'''
# # loading the zip file and creating a zip object 
# with ZipFile("/home/mendu/Thesis/data/magnatagatune/mp3_all.zip", 'r') as zObject: 
#     zObject.extractall(path="/home/mendu/Thesis/data/magnatagatune/audio") 

'To unzip the zipped file of mp3 files'

I ran `find audio -type f -name "*.mp3" -exec mv {} audio \;` to get all the .mp3 files into the audio folder and
`find audio -type d -empty -delete` to delete the empty folders

In [2]:
# Checking the no. of audio files
audio_dir = '/home/mendu/Thesis/data/magnatagatune/audio'
audio_list = os.listdir(audio_dir)
audio_list = [file_name.replace('.wav.mp3', '') for file_name in audio_list] # list of audio track names
print('Total number of audio files', len(audio_list))

Total number of audio files 25863


## Filtering the tags dataframe

In [3]:
#Importing the tags csv file
df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/tag_annotations.csv',sep = "\t" ,index_col=[0])
df['mp3_path'] = [i[2:] for i in df['mp3_path']] 

In [4]:
df

,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,clasical,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
clip_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58899,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-56-la_bressa...
58906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-57-lost_is_m...
58907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-57-lost_is_m...


# For genre

genre: classical, techno, electronic, rock, indian, opera, pop, classic, new age, dance, country, metal

In [5]:
'''genre and synonym_dict for num_class = 8'''
# List of the most frequent genre tags
genre = ['classical', 'clasical', 'classic', 'techno', 'electronic', 'electro', 'electronica', 
 'electric', 'rock', 'indian', 'india', 'opera', 'operatic', 'pop', 'new age', 'dance', 
 'country', 'metal', 'heavy', 'heavy metal']

# Dictionary of genre tags that are synonyms
synonym_dict = {'classical':['classical', 'clasical', 'classic'],
                'electronic': ['electronic', 'electro', 'electronica', 'electric', 'techno', 'dance'],
                'indian': ['indian', 'india'],
                'opera': ['opera', 'operatic'],
                'rock': ['rock', 'metal', 'heavy', 'heavy metal']}

'''genre and synonym_dict for num_class = 10'''
# genre = ['classical', 'clasical', 'classic', 'techno', 'electronic', 'electro', 'electronica', 
#  'electric', 'rock', 'indian', 'india', 'opera', 'operatic', 'pop', 'new age',
#  'country']

# synonym_dict = {'classical':['classical', 'clasical', 'classic'],
#                 'electronic': ['electronic', 'electro', 'electronica', 'electric', 'techno'],
#                 'indian': ['indian', 'india'],
#                 'opera': ['opera', 'operatic'],
#                 'metal': ['metal', 'heavy', 'heavy metal']}


genre.append('mp3_path')

df = df[genre]

In [11]:
df

,mp3_path,tags
clip_id,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,[]
6,american_bach_soloists-j_s__bach_solo_cantatas...,[]
10,american_bach_soloists-j_s__bach_solo_cantatas...,[]
11,american_bach_soloists-j_s__bach_solo_cantatas...,[quiet]
12,american_bach_soloists-j_s__bach_solo_cantatas...,[]
...,...,...
58899,jacob_heringman-blame_not_my_lute-56-la_bressa...,[]
58906,jacob_heringman-blame_not_my_lute-57-lost_is_m...,"[slow, soft]"
58907,jacob_heringman-blame_not_my_lute-57-lost_is_m...,[slow]


In [10]:
# Creating a single column for all the tags
df['tags'] = df.apply(lambda row: (','.join(row.index[row == 1])).split(','), axis=1)

df = df[['mp3_path', 'tags']]

In [8]:
# nested for loop to replace the synonyms
for idx, row in df.iterrows(): #iterates over the rows
    new_tags = []
    for tag in row['tags']: #iterates over each element in the aspect_list list
        for key, values in synonym_dict.items(): #iterates over same_tags dictionary
            if tag in values:
                new_tags.append(key)
                break
        else:
            new_tags.append(tag)
    
    # Making sure the tags are unique in each row        
    new_tags = list(set(new_tags))       
    
    # If multiple tags drop classical if available 
    if len(row['tags'])>1 and 'classical' in row['tags']: #if that are more than one tags and one of it is instrumental drop instrumental
        new_tags.remove('classical')
        
    # updating the tags     
    df.at[idx, 'tags'] = new_tags


# removing empty tags rows
df = df[df['tags'].apply(lambda x: any(x))]

# removing rows with multiple tags
df = df[df['tags'].apply(lambda x: len(x) == 1)]

# converting the values in the tags column from list to str
df['tags'] = df['tags'].apply(lambda x: x[0])


In [9]:
df.tags.value_counts()

tags
electronic    3279
classical     3191
rock          1751
opera         1259
indian        1234
pop            431
country        354
new age        328
Name: count, dtype: int64

## Generating mel-spectrograms

In [10]:
# Function to convert mp3 files to mel-spectrograms
def mp3_to_melspectrogram(input_folder, output_folder, mp3_name_list):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List to store the names of problematic files
    problematic_files = []

    # Iterate over each file in the input folder
    for filename in mp3_name_list:
        try:
            # Load the audio file
            audio_path = os.path.join(input_folder, filename)
            y, sr = librosa.load(audio_path)

            # Compute the mel-spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

            # Convert to decibel scale
            mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Save the mel-spectrogram as .npy file
            output_path = os.path.join(output_folder, filename.replace('.mp3', '.npy'))
            np.save(output_path, mel_spectrogram_db)

            # print(f"Mel-spectrogram saved: {output_path}")
        
        except Exception as e:
            # Handle the exception (e.g., skip the file, log the filename)
            print(f"Error processing {filename}: {str(e)}")
            problematic_files.append(filename)

    # Return the list of problematic files
    return problematic_files

In [11]:
#generating mel-specs
input_folder = audio_dir
output_folder = '/home/mendu/Thesis/data/magnatagatune/mel-specs'
# mp3_to_melspectrogram(input_folder, output_folder, df.mp3_path.tolist())

In [12]:
len(os.listdir('/home/mendu/Thesis/data/magnatagatune/mel-specs'))

11826

In [13]:
# removing the .mp3 file causing problems
problematic_file = 'jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3'

df = df[df['mp3_path'] != problematic_file]

In [14]:
#Checking if there is a mel-spec for every row in the df
df_filenames = df.mp3_path.apply(lambda x: x[:-4]).tolist()
mel_list = [x[:-4] for x in os.listdir(output_folder)]

if sorted(df_filenames) == sorted(mel_list):
    print('There is a mel_spectrogram for every row in the df.')
else:
    print('Every row in the df doesnt have a mel-spectrogram.')

There is a mel_spectrogram for every row in the df.


In [15]:
# Elements in list1 but not in list2
only_in_df = [x for x in df_filenames if x not in mel_list]

# Elements in list2 but not in list1
only_in_mel = [x for x in mel_list if x not in df_filenames]

print("Elements in df_filenames but not in mel_list:", len(only_in_df))
print("Elements in mel_list but not in df_filenames:", len(only_in_mel))

Elements in df_filenames but not in mel_list: 0
Elements in mel_list but not in df_filenames: 0


In [16]:
df

,mp3_path,tags
clip_id,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera
6,american_bach_soloists-j_s__bach_solo_cantatas...,opera
10,american_bach_soloists-j_s__bach_solo_cantatas...,opera
11,american_bach_soloists-j_s__bach_solo_cantatas...,opera
14,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic
...,...,...
58896,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical


In [17]:
# df.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/processed_df.csv')

In [18]:
new_df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/processed_df.csv', index_col=[0])

In [19]:
new_df

,mp3_path,tags
clip_id,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,opera
6,american_bach_soloists-j_s__bach_solo_cantatas...,opera
10,american_bach_soloists-j_s__bach_solo_cantatas...,opera
11,american_bach_soloists-j_s__bach_solo_cantatas...,opera
14,lvx_nova-lvx_nova-01-contimune-30-59.mp3,electronic
...,...,...
58896,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,classical


# For mood

In [5]:
#list of most frequent moods
mood = ['slow', 'fast', 'ambient', 'loud', 'quiet', 'soft', 'weird']

mood.append('mp3_path')

df = df[mood]

In [6]:
df

,slow,fast,ambient,loud,quiet,soft,weird,mp3_path
clip_id,,,,,,,,
2,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
6,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
10,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
11,0,0,0,0,1,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
12,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
...,...,...,...,...,...,...,...,...
58899,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-56-la_bressa...
58906,1,0,0,0,0,1,0,jacob_heringman-blame_not_my_lute-57-lost_is_m...
58907,1,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-57-lost_is_m...


In [7]:
# Creating a single column for all the tags
df['tags'] = df.apply(lambda row: (','.join(row.index[row == 1])).split(','), axis=1)

df = df[['mp3_path', 'tags']]

In [8]:
# removing empty tags rows
df = df[df['tags'].apply(lambda x: any(x))]

# removing rows with multiple tags
df = df[df['tags'].apply(lambda x: len(x) == 1)]

# converting the values in the tags column from list to str
df['tags'] = df['tags'].apply(lambda x: x[0])

In [9]:
df.tags.value_counts()

tags
slow       1985
fast       1793
ambient     849
loud        634
quiet       284
weird       236
soft        217
Name: count, dtype: int64

In [10]:
df

,mp3_path,tags
clip_id,,
11,american_bach_soloists-j_s__bach_solo_cantatas...,quiet
21,lvx_nova-lvx_nova-01-contimune-233-262.mp3,fast
23,lvx_nova-lvx_nova-01-contimune-291-320.mp3,fast
39,american_bach_soloists-heinrich_schutz__musica...,slow
51,american_bach_soloists-heinrich_schutz__musica...,quiet
...,...,...
58800,magnaloops-world_loops_1-43-osabhanschristian5...,slow
58815,jacob_heringman-blame_not_my_lute-45-tantarra_...,soft
58880,magnaloops-world_loops_2-54-osdmichaelmasley64...,ambient


## Generating mel-spectrograms

In [11]:
# Function to convert mp3 files to mel-spectrograms
def mp3_to_melspectrogram(input_folder, output_folder, mp3_name_list):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List to store the names of problematic files
    problematic_files = []

    # Iterate over each file in the input folder
    for filename in mp3_name_list:
        try:
            # Load the audio file
            audio_path = os.path.join(input_folder, filename)
            y, sr = librosa.load(audio_path)

            # Compute the mel-spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

            # Convert to decibel scale
            mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Save the mel-spectrogram as .npy file
            output_path = os.path.join(output_folder, filename.replace('.mp3', '.npy'))
            np.save(output_path, mel_spectrogram_db)

            # print(f"Mel-spectrogram saved: {output_path}")
        
        except Exception as e:
            # Handle the exception (e.g., skip the file, log the filename)
            print(f"Error processing {filename}: {str(e)}")
            problematic_files.append(filename)

    # Return the list of problematic files
    return problematic_files

In [12]:
#generating mel-specs
input_folder = audio_dir
output_folder = '/home/mendu/Thesis/data/magnatagatune/mel-specs-mood'
# problematic_file = mp3_to_melspectrogram(input_folder, output_folder, df.mp3_path.tolist())

In [13]:
len(os.listdir('/home/mendu/Thesis/data/magnatagatune/mel-specs-mood'))

5997

In [14]:
# removing the .mp3 file causing problems
problematic_file = 'norine_braun-now_and_zen-08-gently-117-146.mp3'

df = df[df['mp3_path'] != problematic_file]

In [15]:
#Checking if there is a mel-spec for every row in the df
df_filenames = df.mp3_path.apply(lambda x: x[:-4]).tolist()
mel_list = [x[:-4] for x in os.listdir(output_folder)]

if sorted(df_filenames) == sorted(mel_list):
    print('There is a mel_spectrogram for every row in the df.')
else:
    print('Every row in the df doesnt have a mel-spectrogram.')

There is a mel_spectrogram for every row in the df.


In [16]:
# Elements in list1 but not in list2
only_in_df = [x for x in df_filenames if x not in mel_list]

# Elements in list2 but not in list1
only_in_mel = [x for x in mel_list if x not in df_filenames]

print("Elements in df_filenames but not in mel_list:", len(only_in_df))
print("Elements in mel_list but not in df_filenames:", len(only_in_mel))

Elements in df_filenames but not in mel_list: 0
Elements in mel_list but not in df_filenames: 0


In [17]:
df

,mp3_path,tags
clip_id,,
11,american_bach_soloists-j_s__bach_solo_cantatas...,quiet
21,lvx_nova-lvx_nova-01-contimune-233-262.mp3,fast
23,lvx_nova-lvx_nova-01-contimune-291-320.mp3,fast
39,american_bach_soloists-heinrich_schutz__musica...,slow
51,american_bach_soloists-heinrich_schutz__musica...,quiet
...,...,...
58800,magnaloops-world_loops_1-43-osabhanschristian5...,slow
58815,jacob_heringman-blame_not_my_lute-45-tantarra_...,soft
58880,magnaloops-world_loops_2-54-osdmichaelmasley64...,ambient


In [18]:
# df.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/processed_df_mood.csv')

In [20]:
df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/processed_df_mood.csv')

In [8]:
df

,clip_id,mp3_path,tags
0,11,american_bach_soloists-j_s__bach_solo_cantatas...,quiet
1,21,lvx_nova-lvx_nova-01-contimune-233-262.mp3,fast
2,23,lvx_nova-lvx_nova-01-contimune-291-320.mp3,fast
3,39,american_bach_soloists-heinrich_schutz__musica...,slow
4,51,american_bach_soloists-heinrich_schutz__musica...,quiet
...,...,...,...
5992,58800,magnaloops-world_loops_1-43-osabhanschristian5...,slow
5993,58815,jacob_heringman-blame_not_my_lute-45-tantarra_...,soft
5994,58880,magnaloops-world_loops_2-54-osdmichaelmasley64...,ambient
5995,58907,jacob_heringman-blame_not_my_lute-57-lost_is_m...,slow


# Adding the caption_embedding column

In [21]:
# Define the Encoder class focusing only on the encoding part
class Encoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Encoder, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.BatchNorm1d(128),
            nn.Linear(128, encoding_size),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.layers(x)
    
# Define input and encoding sizes
input_size = 768
encoding_size = 128
# encoding_size = 64
def load_encoder_state_dict(encoder, state_dict_path):
    state_dict = torch.load(state_dict_path)
    # # Rename the state_dict keys to match the structure of the Encoder class
    # new_state_dict = OrderedDict()
    # for k, v in state_dict.items():
    #     # Prepend 'layers.' to each key
    #     name = f'layers.{k}'
    #     new_state_dict[name] = v
    
    # encoder.load_state_dict(new_state_dict)
    # return encoder
    # Filter out unexpected keys (e.g., BatchNorm layers not present in the current model)
    filtered_state_dict = {k: v for k, v in state_dict.items() if k in encoder.state_dict()}
    
    # Load the filtered state_dict
    encoder.load_state_dict(filtered_state_dict, strict=False)  # strict=False allows for size mismatches
    return encoder

def get_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device = get_device()
# Instantiate the encoder
encoder = Encoder(input_size=input_size, encoding_size=encoding_size).to(device)
encoder.eval()  # Set the encoder to evaluation mode

# Load the pre-trained weights for the encoder for 64 embedding vector
encoder = load_encoder_state_dict(encoder, '/home/mendu/Thesis/data/musiccaps/auto_encoder/encoder_state_dict.pth')

# Load the pre-trained weights for the encoder for 128 embedding vector
# encoder = load_encoder_state_dict(encoder, '/home/mendu/Thesis/data/musiccaps/auto_encoder/encoder_state_dict128.pth')

# Load the SentenceTransformer model and move to the correct device
roberta_model = SentenceTransformer('/home/mendu/Thesis/data/musiccaps/new_embedding_model').to(device)

# Assuming df is a predefined DataFrame with text data
# caption = df.caption_writing[2]
# Function to encode caption
def encode_caption(encoder, sentence_model, text):
    with torch.no_grad():
        encoded_caption = sentence_model.encode(text, convert_to_tensor=True)
        encoded_caption = encoded_caption.to(device)
        return encoder(encoded_caption.unsqueeze(0))
    
# Encoded caption
# fully_encoded_caption = encode_caption(encoder, roberta_model, caption)

In [22]:
import pandas as pd
test_caption = pd.read_parquet('/home/mendu/Thesis/data/magnatagatune/captions/test-00000-of-00001-94781ef88fa7ed89.parquet', engine='pyarrow', columns=['track_id', 'caption_writing', 'path'])
train_caption = pd.read_parquet('/home/mendu/Thesis/data/magnatagatune/captions/train-00000-of-00001-28dbf9154d6d526d.parquet', engine='pyarrow', columns=['track_id', 'caption_writing', 'path'])
valid_caption = pd.read_parquet('/home/mendu/Thesis/data/magnatagatune/captions/valid-00000-of-00001-bf9893b31ca2d5e5.parquet', engine='pyarrow', columns=['track_id', 'caption_writing', 'path'])
df_caption = pd.concat([test_caption, train_caption, valid_caption], ignore_index = True)
df_caption['path'] = df_caption['path'].apply(lambda i: i[2:])

In [21]:
# df_caption.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_captions.csv')

In [23]:
# Apply the encode_caption function to each row's caption_writing
def get_encoded_caption(row):
    caption = row['caption_writing']
    return encode_caption(encoder, roberta_model, caption).cpu()


df_caption['caption_embedding'] = df_caption.apply(get_encoded_caption, axis=1)

# Flatten the nested list
df_caption['caption_embedding'] = df_caption['caption_embedding'].apply(lambda x: np.array(x)[0])

In [35]:
df_caption.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_capemb_all.csv')

In [24]:
df_caption['track_id'] = df_caption['track_id'].astype(int)
df_merge = pd.merge(df, df_caption, how = 'inner', left_on='clip_id', right_on='track_id')
df_merge

,clip_id,mp3_path,tags,track_id,caption_writing,path,caption_embedding
0,11,american_bach_soloists-j_s__bach_solo_cantatas...,quiet,11,This atmospheric and introspective song blends...,american_bach_soloists-j_s__bach_solo_cantatas...,"[0.0, 0.069062, 0.0, 0.0, 0.10199389, 0.017180..."
1,21,lvx_nova-lvx_nova-01-contimune-233-262.mp3,fast,21,This high-energy song merges electronic elemen...,lvx_nova-lvx_nova-01-contimune-233-262.mp3,"[0.0, 0.08048035, 0.0, 0.0, 0.0926192, 0.00765..."
2,23,lvx_nova-lvx_nova-01-contimune-291-320.mp3,fast,23,Get ready to race to the top of the charts wit...,lvx_nova-lvx_nova-01-contimune-291-320.mp3,"[0.0, 0.058119245, 0.0, 0.011021233, 0.0871724..."
3,39,american_bach_soloists-heinrich_schutz__musica...,slow,39,This heartfelt ballad takes the listener on a ...,american_bach_soloists-heinrich_schutz__musica...,"[0.0, 0.06326667, 0.0, 0.0, 0.098317504, 0.023..."
4,51,american_bach_soloists-heinrich_schutz__musica...,quiet,51,Experience the enchantment of a classical duet...,american_bach_soloists-heinrich_schutz__musica...,"[0.0, 0.057540625, 0.0, 0.0044447742, 0.093889..."
...,...,...,...,...,...,...,...
5992,58800,magnaloops-world_loops_1-43-osabhanschristian5...,slow,58800,This haunting and introspective instrumental t...,magnaloops-world_loops_1-43-osabhanschristian5...,"[0.0, 0.06094677, 0.0, 0.00035023037, 0.094731..."
5993,58815,jacob_heringman-blame_not_my_lute-45-tantarra_...,soft,58815,This enchanting instrumental composition beaut...,jacob_heringman-blame_not_my_lute-45-tantarra_...,"[0.0, 0.05936481, 0.0, 0.0057750293, 0.0870878..."
5994,58880,magnaloops-world_loops_2-54-osdmichaelmasley64...,ambient,58880,This ambient track creates a dreamy atmosphere...,magnaloops-world_loops_2-54-osdmichaelmasley64...,"[0.0, 0.05700565, 0.0, 0.0, 0.11397909, 0.0155..."
5995,58907,jacob_heringman-blame_not_my_lute-57-lost_is_m...,slow,58907,This beautiful classical piece features a gent...,jacob_heringman-blame_not_my_lute-57-lost_is_m...,"[0.0, 0.058866613, 0.0, 0.0037290733, 0.081891..."


In [25]:
(df_merge['mp3_path'] == df_merge['path']).value_counts()

True    5997
Name: count, dtype: int64

In [26]:
df_merge = df_merge[['clip_id', 'mp3_path', 'tags', 'caption_embedding']]

In [27]:
df_merge.to_pickle('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_mood_128embeddings.pkl')